In [26]:
import os
import json
import sqlite3
import anthropic

import numpy as np
import pandas as pd

from dotenv import load_dotenv
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
load_dotenv()
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")


In [20]:
class AnthropicCalls:
    def __init__(
            self,
            name="Anthropic Chat",
            api_key="",
            model="claude-3-5-sonnet-20240620",
            max_tokens=1024,
            temperature=0.7,
            system_prompt="",
            stream=False,
    ):
        self.name = name
        self.api_key = api_key
        self.model = model
        self.max_tokens = max_tokens
        self.temperature = temperature
        self.system_prompt = system_prompt
        self.stream = stream
        self.history = []

        self.client = anthropic.Anthropic(
            api_key=self.api_key,
        )

    def add_message(self, role, content):
        self.history.append(
            {
                "role": role, 
                "content": content
            }
        )
    
    def clear_history(self):
        self.history.clear()

    def chat(self, message, **kwargs):
        self.add_message("user", message)
        return self.get_response(**kwargs)
        
    def get_response(self, should_print=True, **kwargs):
        params = {
            "model": self.model,
            "max_tokens": self.max_tokens,
            "temperature": self.temperature,
            "messages": self.history,
            "system": self.system_prompt,
            **kwargs
        }

        if self.stream:
            assistant_response = ""
            with self.client.messages.stream(
                **params
            ) as stream:
                for text_chunk in stream.text_stream:
                    assistant_response += str(text_chunk)
                    if should_print:
                        print(text_chunk, end="", flush=True)
        else:
            response = self.client.messages.create(
                **params
            )
            assistant_response = str(response.content[0].text)
            if should_print:
                print(assistant_response, end="")

        if should_print:
            print()

        self.add_message("assistant", assistant_response)
        return assistant_response



In [ ]:
calls = AnthropicCalls(api_key=ANTHROPIC_API_KEY)

calls.chat(
    message="Hi!",
)


In [18]:
calls = AnthropicCalls(api_key=ANTHROPIC_API_KEY, stream=True)

calls.chat(
    message="Write a Long abstract",
)


Here's a long abstract for a hypothetical research paper:

Title: "The Impact of Social Media Usage on Adolescent Mental Health: A Longitudinal Study"

Abstract:

In recent years, the pervasive use of social media platforms among adolescents has raised concerns about its potential effects on mental health and well-being. This longitudinal study aimed to investigate the relationship between social media usage patterns and various mental health indicators among adolescents aged 13-18 over a three-year period. The research addressed several key questions: (1) How does the frequency and duration of social media use correlate with symptoms of anxiety, depression, and low self-esteem? (2) Are there differences in mental health outcomes based on the types of social media platforms used? (3) What role do factors such as age, gender, and socioeconomic status play in moderating the relationship between social media use and mental health?

The study employed a mixed-methods approach, combining qu

'Here\'s a long abstract for a hypothetical research paper:\n\nTitle: "The Impact of Social Media Usage on Adolescent Mental Health: A Longitudinal Study"\n\nAbstract:\n\nIn recent years, the pervasive use of social media platforms among adolescents has raised concerns about its potential effects on mental health and well-being. This longitudinal study aimed to investigate the relationship between social media usage patterns and various mental health indicators among adolescents aged 13-18 over a three-year period. The research addressed several key questions: (1) How does the frequency and duration of social media use correlate with symptoms of anxiety, depression, and low self-esteem? (2) Are there differences in mental health outcomes based on the types of social media platforms used? (3) What role do factors such as age, gender, and socioeconomic status play in moderating the relationship between social media use and mental health?\n\nThe study employed a mixed-methods approach, co

In [21]:
calls = AnthropicCalls(api_key=ANTHROPIC_API_KEY, stream=True)

message = ""
while message != "END": 
    message = input("User:")
    calls.chat(message=message)


Hello! How can I assist you today? Feel free to ask me any questions or let me know if you need help with anything.
As an AI language model, I don't have feelings or personal experiences, but I'm functioning well and ready to assist you with any questions or tasks you might have. How can I help you today?
Certainly! Photons are fascinating particles that play a crucial role in our understanding of light and electromagnetic radiation. Here are some key ideas and facts about photons:

1. Definition: Photons are elementary particles that represent the quantum of light and all other forms of electromagnetic radiation.

2. Wave-particle duality: Photons exhibit both wave-like and particle-like properties, demonstrating the concept of wave-particle duality in quantum mechanics.

3. Mass and charge: Photons have no mass and no electric charge.

4. Speed: In a vacuum, photons always travel at the speed of light (approximately 299,792,458 meters per second).

5. Energy: The energy of a photon i

In [27]:
class SQLiteCalls:
    def __init__(
            self,
            db_path="sqlite.db"
    ):
        self.db_path = db_path

    def setup_database(self):
        conn = sqlite3.connect(self.db_path)

        # cursor = conn.cursor()
        # cursor.execute('DROP TABLE IF EXISTS chat_history')
        
        conn.execute("""
            CREATE TABLE IF NOT EXISTS chat_history (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                role TEXT,
                message TEXT,
                embeddings TEXT,
                date TEXT DEFAULT CURRENT_TIMESTAMP,
            );
        """)
        conn.commit()
        conn.close()

    def save_message(self, message: dict, embeddings):
        conn = sqlite3.connect(self.db_path, timeout=5)
        try:
            cursor = conn.cursor()
            role = message["role"]
            message_str = json.dumps(message)
            embeddings_str = json.dumps(embeddings) if not isinstance(embeddings, str) else embeddings

            cursor.execute("""
                INSERT INTO chat_history (role, message, embeddings)
                VALUES (?, ?, ?)
            """, (role, message_str, embeddings_str))

            conn.commit()
        finally:
            conn.close()

    def load_chat_to_dataframe(self):
        conn = sqlite3.connect(self.db_path, timeout=5)
        df = pd.read_sql_query('''
            SELECT * FROM chat_history
            ORDER BY date ASC                
        ''', conn)
        
        conn.close()
        return df
